In [1]:
import requests 
from bs4 import BeautifulSoup

In [2]:
url_base = "https://www.usine-digitale.fr"

In [3]:
# Cette fonction permet de recupérer tout le texte qui se retrouve dans une balise. 
def get_text (soup_obj) :
    all_string=""
    for node in soup_obj:
        all_string = all_string + ''.join(node.findAll(text=True))
    return all_string.replace("\n", "").replace("\t", "").replace("\r", " ").strip()



# Cette fonction permet de récupérer les données 
def get_all_data_companies (link): 

    r_page = requests.get(link) 
    soup_page = BeautifulSoup(r_page.text, 'html.parser')
        
    # On récupère toutes les balises <a> qui contiennent les liens des entreprises
    results_details_links = soup_page.select('section.blocType1 a.contenu')
    
        
    links_company =  [] # Contiendra les liens qu'on aura extrait des balises <a> précédentes
    for i in range (0, len(results_details_links)) :
    
        # On extrait le lien dans l'attribut "href" de chaque balise <a> et on ajoute la base : "https://www.usine-digitale.fr"
        link = url_base+results_details_links[i]["href"].strip()
        
        # On injecte le lien dans le tableau
        links_company.append(link)
    
    list_companies_data =[] 
    # On recupère les données dans chaque page de detail d'une entreprise
    for j in range (0, len(links_company)):
        
        r_company_details = requests.get(links_company[j])

        # On formate en type html
        soup_company_details = BeautifulSoup(r_company_details.text, "html.parser")
        
        # On extrait les données de l'enteprise séleectionnée
        try:#### Test des opérations
            name = get_text(soup_company_details.select("h1[itemprop = name]"))
            presentation =get_text(soup_company_details.select("div[itemprop = description]"))
            dateCreation =get_text(soup_company_details.select("meta[itemprop = foundingDate] + p"))
            email = get_text(soup_company_details.select("p[itemprop = email]"))
            address  = get_text(soup_company_details.select("p[itemprop = address]"))
            telephone = get_text(soup_company_details.select("p[itemprop = telephone]"))
            genese =get_text(soup_company_details.select("div[itemprop = review] > p"))
            produits =get_text(soup_company_details.select("div[itemprop = makesOffer] > p"))
            createur =get_text(soup_company_details.select("div[itemprop = founders] > p"))
            implantation =get_text(soup_company_details.select("div[itemprop = address] > p"))
            
            list_companies_data.append((name,presentation,dateCreation,email,address,telephone,genese,produits,createur,implantation))
            
        except: # S'il y a un problème, on passe à la prochaine valeur
            continue
    
    # Création du fichier dataset
    import pandas as pd
    df = pd.DataFrame(list_companies_data, columns=["name","presentation","dateCreation","email","address","telephone","genese","produits","createur","implantation"])
    df.to_csv('companies_data.csv', index=False, encoding='utf-8')

In [4]:
r = requests.get(url_base+"/annuaire-start-up") # Récupération de la page principale qui contient les annuaires

soup = BeautifulSoup(r.text, 'html.parser') # On formate le contenu html de type texte vers le type html

nb_page = 2 # Nombre de pages souhaitées (maxi 9)

results = soup.select('ul.pagination li a') # Récupération des balises qui contiennent les liens de pagination

if (nb_page <= 9 and nb_page >=0):
    
    # On parcours les liens de pagination
    for i in range (0, nb_page) :
        link_page = url_base+results[i]["href"].strip() # lien de la page courante

        get_all_data_companies(link_page) # Récupération des données de chaque page
        

